# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [ ]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


In [ ]:
sum([i for i in square_root_generator(5)])

In [ ]:
generator = square_root_generator(limit=13)
print([i for i in generator][-1])

# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [1]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [2]:
import dlt

In [3]:
import duckdb

In [4]:
# define the connection to load to.
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

In [5]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

In [8]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")
people = conn.sql("SELECT * FROM generators.people").df()
display(people)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707756147.5242994,m4Uu2Mx7StwR9Q,None
1,2,Person_2,27,City_A,1707756147.5242994,pxTowza6Ars1gQ,None
2,3,Person_3,28,City_A,1707756147.5242994,DcgXk8ZLjp3Oig,None
3,4,Person_4,29,City_A,1707756147.5242994,bsUc0mXfcQx2lw,None
4,5,Person_5,30,City_A,1707756147.5242994,gocrlEUdJINeGQ,None
5,3,Person_3,33,City_B,1707756148.1439703,Ci7N4POinY9J7g,Job_3
6,4,Person_4,34,City_B,1707756148.1439703,ZEocLg0kzx9/Nw,Job_4
7,5,Person_5,35,City_B,1707756148.1439703,IxeSqLAH9GvKXg,Job_5
8,6,Person_6,36,City_B,1707756148.1439703,coAS8LAEkRWfnw,Job_6
9,7,Person_7,37,City_B,1707756148.1439703,vnNXmLFmqXn1cQ,Job_7


In [9]:
f"Appended sum of ages: {people.age.sum()}"

'Appended sum of ages: 353'

In [10]:
# use merge write
merged_generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='merged_generators', )


# we can load any generator to a table at the pipeline destnation as follows:
info = merged_generators_pipeline.run(people_1(),
										table_name="people_merged",
										write_disposition="merge", primary_key='ID')

info = merged_generators_pipeline.run(people_2(),
										table_name="people_merged",
										write_disposition="merge", primary_key='ID')

In [11]:
conn = duckdb.connect(f"{merged_generators_pipeline.pipeline_name}.duckdb")
people_merged = conn.sql("SELECT * FROM merged_generators.people_merged").df()
display(people_merged)

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,2,Person_2,27,City_A,1707756203.9930103,MLUzhKRxplLvvA,None
1,1,Person_1,26,City_A,1707756203.9930103,3bvkLzrDKutRSg,None
2,8,Person_8,38,City_B,1707756204.695488,UjjSDH70Lla9ZQ,Job_8
3,5,Person_5,35,City_B,1707756204.695488,03RE/6XXMj/XNw,Job_5
4,4,Person_4,34,City_B,1707756204.695488,gQXk0QYklCdntA,Job_4
5,7,Person_7,37,City_B,1707756204.695488,7tSoq/n8I9e0AA,Job_7
6,3,Person_3,33,City_B,1707756204.695488,vltfSJyhhuMgFA,Job_3
7,6,Person_6,36,City_B,1707756204.695488,dgHFqSXngOC3rA,Job_6


In [12]:
f"Appended sum of ages: {people_merged.age.sum()}"

'Appended sum of ages: 266'